In [2]:
import tensorflow as tf
import math
import tensorflow.python.keras.backend as K
from tensorflow.python.keras.layers import UpSampling2D
from tensorflow.python.keras.activations import relu
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import DepthwiseConv2D 
from tensorflow.python.keras.layers import MaxPooling2D
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import merge
from tensorflow.python.keras import Input
from tensorflow.python.keras.backend import sigmoid
from tensorflow.python.keras.optimizers import SGD, Adam
from tensorflow.python.keras.layers import Add
from tensorflow.python.keras.layers import Multiply
from tensorflow.python.keras.layers import ZeroPadding2D
from tensorflow.python.keras.layers import BatchNormalization as bn
from tensorflow.python.keras.layers import GlobalAveragePooling2D
from tensorflow.python.keras.layers import Concatenate
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import np_utils



AttributeError: module 'tensorflow' has no attribute 'python'

In [0]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
def ew(p):
    return int(1.1*p)
def sliced(x,i,j):
        return x[:,:,:,:i-j]

In [0]:
def ghostmodule(x,exp,ratio=2):
    init_channels=math.ceil(exp / ratio)
    x=Conv2D(init_channels,(1,1))(x)
    dw1=DepthwiseConv2D((1,1))(x)
    dw2=Lambda(lambda t:sliced(t,exp,init_channels))(dw1)
    x=Concatenate(axis=3)([x,dw2])
    return x

    
    

In [0]:
def mbconv1a(m,out):
  m = DepthwiseConv2D((3,3),padding='same')(m)
  m=bn(momentum=0.99)(m)
  m=Activation('swish')(m)
  m = ghostmodule(m,ew(out))
  m=bn(momentum=0.99)(m)
  return Activation('swish')(m)

    
    
def mbconv(x, expand,squeeze):
  expand=K.int_shape(x)[3]*6
  if K.int_shape(x)[3]==ew(squeeze):
    y=x
  else:
   y=ghostmodule(x,ew(squeeze))
   y=bn(momentum=0.99)(y)
  m = ghostmodule(x,ew(expand))
  m=bn(momentum=0.99)(m)
  m=Activation('swish')(m)
  m = DepthwiseConv2D((3,3),padding='same')(m)
  m=bn(momentum=0.99)(m)
  m=Activation('swish')(m)
  m = ghostmodule(m,ew(squeeze))
  m=bn(momentum=0.99)(m)
  final=Add()([m, y])
  return Activation('swish')(final)
   

def mbconv2a(x, expand, squeeze):
  expand=K.int_shape(x)[3]*6
  if K.int_shape(x)[3]==ew(squeeze):
    y=x
  else:
    y=ghostmodule(x,ew(squeeze))
    y=bn(momentum=0.99)(y)
  m = ghostmodule(x,ew(expand))
  m = DepthwiseConv2D((5,5),padding='same')(m)
  m=bn(momentum=0.99)(m)
  m=Activation('swish')(m)
  m=se_block(m,K.int_shape(m)[3])
  m = ghostmodule(m,ew(squeeze))
  m=bn(momentum=0.99)(m)
  final=Add()([m, y])
  return Activation('swish')(final)
  


def mbconv61(t,expand,squeeze,k,stride):
    expand=K.int_shape(t)[3]*6
    b=ghostmodule(t,ew(expand))
    b=bn(momentum=0.99)(b)
    b=Activation('swish')(b)
    c=Conv2D(ew(squeeze),(1,1),strides=(stride,stride))(t)
    c=bn(momentum=0.99)(c)
    b=ZeroPadding2D(1)(b)
    b=DepthwiseConv2D((k,k),strides=(stride,stride))(b)
    b=bn(momentum=0.99)(b)
    b=Activation('swish')(b)
    b=ghostmodule(b,ew(squeeze))
    b=bn(momentum=0.99)(b)
    d=Add()([b,c])
    return Activation('swish')(d)
       
def mbconv62(t,expand,squeeze,k,stride):
    expand=K.int_shape(t)[3]*6
    b=ghostmodule(t,ew(expand))
    b=bn(momentum=0.99)(b)
    b=Activation('swish')(b)
    c=Conv2D(ew(squeeze),(1,1),strides=(stride,stride))(t)
    c=bn(momentum=0.99)(c)
    b=ZeroPadding2D(2)(b)
    b=DepthwiseConv2D((k,k),strides=(stride,stride))(b)
    b=bn(momentum=0.99)(b)
    b=Activation('swish')(b)
    b=se_block(b,K.int_shape(b)[3])
    b=ghostmodule(b,ew(squeeze))
    b=bn(momentum=0.99)(b)
    d=Add()([b,c])
    return Activation('swish')(d)
    
     
def se_block(in_block, ch, ratio=4):
    x = GlobalAveragePooling2D()(in_block)
    x = Dense(ch//ratio, activation='swish')(x)
    x = Dense(ch, activation='sigmoid')(x)
    return Multiply()([in_block,x])



In [0]:
padding=ZeroPadding2D(1)(input)
conv1=Conv2D(32,(3,3),activation='swish',strides=2,use_bias=False)(padding)
mbconv1=mbconv1a(conv1,16)
mbconv2=mbconv(mbconv1,40,20)
mbconv3=mbconv61(mbconv2,48,24,3,2)
mbconv4=mbconv2a(mbconv3,64,32)
mbconv5=mbconv62(mbconv4,80,40,5,2)
mbconv6=mbconv(mbconv5,80,40)
mbconv7=mbconv(mbconv6,120,60)
mbconv8=mbconv2a(mbconv7,160,80)
mbconv9=mbconv2a(mbconv8,160,128)
mbconv9=mbconv2a(mbconv9,160,160)
mbconv10=mbconv2a(mbconv9,192,256)
mbconv11=mbconv2a(mbconv10,224,256)
mbconv12=mbconv2a(mbconv11,264,512)
mbconv13=mbconv(mbconv12,304,1024)
psp1=AveragePooling2D((32,32))(mbconv13)
psp2=AveragePooling2D((16,16))(mbconv13)
psp3=AveragePooling2D((10,10))(mbconv13)
psp4=AveragePooling2D((5,5))(mbconv13)
psp1=Conv2D(256,(1,1))(psp1)
psp1=bn(momentum=0.99)(psp1)
psp1=UpSampling2D((32,32),interpolation='bilinear')(psp1)
psp2=Conv2D(256,(1,1))(psp2)
psp2=UpSampling2D((16,16),interpolation='bilinear')(psp2)
psp3=Conv2D(256,(1,1))(psp3)
psp3=UpSampling2D((10,10),interpolation='bilinear')(psp3)
psp3=ZeroPadding2D(1)(psp3)
psp4=Conv2D(256,(1,1))(psp4)
psp4=UpSampling2D((5,5),interpolation='bilinear')(psp4)
psp4=ZeroPadding2D(1)(psp4)
concat1=Concatenate()([psp1,psp2,psp3,psp4])
concat2=Concatenate()([concat1,mbconv13])
final=Conv2D(100,(1,1))(concat2)
final=bn(momentum=0.99)(final)
final=Activation('relu')(final)
final=UpSampling2D((2,2),interpolation='bilinear')(final)
final=mbconv(final,80,100)
final=UpSampling2D((2,2),interpolation='bilinear')(final)
final=mbconv(final,80,100)
final=UpSampling2D((2,2),interpolation='bilinear')(final)
final=mbconv(final,80,100)
final=Activation('softmax')(final)





In [0]:
opt = tf.keras.optimizers.Adam()
model=Model(inputs=input,outputs=mbconv7)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 258, 258, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_138 (Conv2D)             (None, 128, 128, 32) 864         zero_padding2d_19[0][0]          
__________________________________________________________________________________________________
depthwise_conv2d_169 (Depthwise (None, 128, 128, 32) 320         conv2d_138[0][0]                 
____________________________________________________________________________________________

In [0]:
num_classes=10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(x_train)
model_checkpoint = ModelCheckpoint("net_weights_{epoch:03d}.hdf5", monitor='accuracy', verbose=1, 
                                   save_best_only=True, save_weights_only=True, period = 5)

callbacks_list = [model_checkpoint]
model.fit(datagen.flow(x_train,y_train,batch_size=32), callbacks = callbacks_list,
                    steps_per_epoch = len(x_train)/32, epochs = 100)






Epoch 1/100


ValueError: ignored

In [0]:
print(model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False))
